<a href="https://colab.research.google.com/github/telmacarvalho/pi-4-univesp/blob/main/treinamento_ml_pi_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Treinamento do Modelo Multi-Target

## Importação de bibliotecas

In [146]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import joblib
import json
import os

## Importação dos dados

In [147]:
# Autoriza acesso ao Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [148]:
# Extrai o dfs da pasta refined
file_path = '/content/drive/MyDrive/dados/csv/refined/data_processed.csv'
df_processed = pd.read_csv(file_path)
df_processed.head()

,cnae,total_acidentes,com_cat_registrada,acidentes_tipicos,acidentes_trajeto,doenca_trabalho,sem_cat_registrada,ano,total_acidentes_seguinte,com_cat_registrada_seguinte,acidentes_tipicos_seguinte,acidentes_trajeto_seguinte,doenca_trabalho_seguinte,sem_cat_registrada_seguinte
0,910,1161,1104,900,141,63,57,2014,828,828,708,110,10,0
1,910,828,828,708,110,10,0,2015,562,525,454,56,15,37
2,910,562,525,454,56,15,37,2016,405,366,307,47,12,39
3,910,405,366,307,47,12,39,2017,517,479,408,45,26,38
4,910,517,479,408,45,26,38,2018,391,369,319,43,7,22


## 1. Divisão dos dados para treinamento

In [149]:
# Define as colunas atributos
colunas_alvo = [
    'total_acidentes',
    'com_cat_registrada',
    'acidentes_tipicos',
    'acidentes_trajeto',
    'doenca_trabalho',
    'sem_cat_registrada'
]

In [150]:
# Seleção dos Atributos (X)
X_data = df_processed[colunas_alvo]
X_data.head()

,total_acidentes,com_cat_registrada,acidentes_tipicos,acidentes_trajeto,doenca_trabalho,sem_cat_registrada
0,1161,1104,900,141,63,57
1,828,828,708,110,10,0
2,562,525,454,56,15,37
3,405,366,307,47,12,39
4,517,479,408,45,26,38


In [151]:
# Seleção dos Targets (y)
y_data = df_processed[[col + '_seguinte' for col in colunas_alvo]]
y_data.head()

,total_acidentes_seguinte,com_cat_registrada_seguinte,acidentes_tipicos_seguinte,acidentes_trajeto_seguinte,doenca_trabalho_seguinte,sem_cat_registrada_seguinte
0,828,828,708,110,10,0
1,562,525,454,56,15,37
2,405,366,307,47,12,39
3,517,479,408,45,26,38
4,391,369,319,43,7,22


In [152]:
print("Dimensões de X (atributos):", X_data.shape)
print("Dimensões de y (alvos múltiplos):", y_data.shape)

Dimensões de X (atributos): (90, 6)
Dimensões de y (alvos múltiplos): (90, 6)


## Treinamento do modelo

In [153]:
df_processed.ano.unique()

array([2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022])

In [154]:
# Divisão temporal dos dados
train_indices = df_processed[df_processed['ano'] <= 2021].index
test_indices = df_processed[df_processed['ano'] == 2022].index

X_train, X_test = X_data.loc[train_indices], X_data.loc[test_indices]
y_train, y_test = y_data.loc[train_indices], y_data.loc[test_indices]

In [155]:
# Inicializa e treina o modelo
modelo_multi = RandomForestRegressor(n_estimators=100, random_state=42)
modelo_multi.fit(X_train, y_train)

print("Modelo de Regressão Multi-alvo treinado com sucesso!")

Modelo de Regressão Multi-alvo treinado com sucesso!


## 3. Avaliação do Modelo

In [156]:
X_test.columns

Index(['total_acidentes', 'com_cat_registrada', 'acidentes_tipicos',
       'acidentes_trajeto', 'doenca_trabalho', 'sem_cat_registrada'],
      dtype='object')

In [157]:
X_test.shape

(10, 6)

*As previsões serão um array com 6 colunas*

In [158]:
# Faz as previsões
y_pred = modelo_multi.predict(X_test)
y_pred.shape

(10, 6)

In [159]:
# Cria um df com as previsões
df_pred = pd.DataFrame(y_pred, columns=y_test.columns, index=y_test.index)
df_pred.shape

(10, 6)

In [160]:
# Itera sobre cada coluna de alvo para calcular as métricas individualmente
metricas = {}
for i, col_name in enumerate(y_test.columns):
    mae = mean_absolute_error(y_test[col_name], df_pred[col_name])
    r2 = r2_score(y_test[col_name], df_pred[col_name])

    metricas[col_name.replace('_seguinte', '').upper()] = {
        'Erro Medio Absoluto (MAE)': mae,
        'Coeficiente R2': r2
    }

    print(f"--- {col_name.replace('_seguinte', '').upper()} ---")
    print(f"  Erro Médio Absoluto (MAE): {mae:.2f}")
    print(f"  Coeficiente R²: {r2:.2f}\n")

--- TOTAL_ACIDENTES ---
  Erro Médio Absoluto (MAE): 429.22
  Coeficiente R²: 0.83

--- COM_CAT_REGISTRADA ---
  Erro Médio Absoluto (MAE): 378.42
  Coeficiente R²: 0.85

--- ACIDENTES_TIPICOS ---
  Erro Médio Absoluto (MAE): 317.39
  Coeficiente R²: 0.83

--- ACIDENTES_TRAJETO ---
  Erro Médio Absoluto (MAE): 84.41
  Coeficiente R²: 0.85

--- DOENCA_TRABALHO ---
  Erro Médio Absoluto (MAE): 13.94
  Coeficiente R²: 0.62

--- SEM_CAT_REGISTRADA ---
  Erro Médio Absoluto (MAE): 50.99
  Coeficiente R²: 0.46



In [161]:
# Busca as colunas 'cnae' e 'ano' do DataFrame original usando o índice do conjunto de teste
identificadores = df_processed.loc[y_test.index, ['cnae', 'ano']]
identificadores.head()

,cnae,ano
8,910,2022
17,1921,2022
26,2513,2022
35,3321,2022
44,3520,2022


In [162]:
df_pred.head(2)

,total_acidentes_seguinte,com_cat_registrada_seguinte,acidentes_tipicos_seguinte,acidentes_trajeto_seguinte,doenca_trabalho_seguinte,sem_cat_registrada_seguinte
8,355.75,327.23,262.46,33.75,31.02,28.52
17,144.23,111.84,75.30,29.78,6.76,32.39


In [163]:
# Converte colunas para int
df_pred = df_pred.astype(int)
df_pred.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 8 to 89
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype
---  ------                       --------------  -----
 0   total_acidentes_seguinte     10 non-null     int64
 1   com_cat_registrada_seguinte  10 non-null     int64
 2   acidentes_tipicos_seguinte   10 non-null     int64
 3   acidentes_trajeto_seguinte   10 non-null     int64
 4   doenca_trabalho_seguinte     10 non-null     int64
 5   sem_cat_registrada_seguinte  10 non-null     int64
dtypes: int64(6)
memory usage: 560.0 bytes


In [164]:
# Renomea as colunas de previsão
y_pred_renomeado = df_pred.rename(columns=lambda c: c.replace('_seguinte', '_previsto'))
y_pred_renomeado.columns

Index(['total_acidentes_previsto', 'com_cat_registrada_previsto',
       'acidentes_tipicos_previsto', 'acidentes_trajeto_previsto',
       'doenca_trabalho_previsto', 'sem_cat_registrada_previsto'],
      dtype='object')

In [165]:
# Junta tudo em um único df de resultados
df_resultados_completos = pd.concat([identificadores, y_test, y_pred_renomeado], axis=1)
df_resultados_completos.head(2)

,cnae,ano,total_acidentes_seguinte,com_cat_registrada_seguinte,acidentes_tipicos_seguinte,acidentes_trajeto_seguinte,doenca_trabalho_seguinte,sem_cat_registrada_seguinte,total_acidentes_previsto,com_cat_registrada_previsto,acidentes_tipicos_previsto,acidentes_trajeto_previsto,doenca_trabalho_previsto,sem_cat_registrada_previsto
8,910,2022,490,450,357,62,31,40,355,327,262,33,31,28
17,1921,2022,265,205,111,72,22,60,144,111,75,29,6,32


In [166]:
df_resultados_completos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 8 to 89
Data columns (total 14 columns):
 #   Column                       Non-Null Count  Dtype
---  ------                       --------------  -----
 0   cnae                         10 non-null     int64
 1   ano                          10 non-null     int64
 2   total_acidentes_seguinte     10 non-null     int64
 3   com_cat_registrada_seguinte  10 non-null     int64
 4   acidentes_tipicos_seguinte   10 non-null     int64
 5   acidentes_trajeto_seguinte   10 non-null     int64
 6   doenca_trabalho_seguinte     10 non-null     int64
 7   sem_cat_registrada_seguinte  10 non-null     int64
 8   total_acidentes_previsto     10 non-null     int64
 9   com_cat_registrada_previsto  10 non-null     int64
 10  acidentes_tipicos_previsto   10 non-null     int64
 11  acidentes_trajeto_previsto   10 non-null     int64
 12  doenca_trabalho_previsto     10 non-null     int64
 13  sem_cat_registrada_previsto  10 non-null     int64
dtypes

In [167]:
# Renomea as colunas
df_resultados_completos = df_resultados_completos.rename(columns=lambda c: c.replace('_seguinte', ''))
df_resultados_completos.columns

Index(['cnae', 'ano', 'total_acidentes', 'com_cat_registrada',
       'acidentes_tipicos', 'acidentes_trajeto', 'doenca_trabalho',
       'sem_cat_registrada', 'total_acidentes_previsto',
       'com_cat_registrada_previsto', 'acidentes_tipicos_previsto',
       'acidentes_trajeto_previsto', 'doenca_trabalho_previsto',
       'sem_cat_registrada_previsto'],
      dtype='object')

In [168]:
# Adequa o ano
df_resultados_completos['ano'] = df_resultados_completos['ano'].replace(2022,2023)
df_resultados_completos.head()

,cnae,ano,total_acidentes,com_cat_registrada,acidentes_tipicos,acidentes_trajeto,doenca_trabalho,sem_cat_registrada,total_acidentes_previsto,com_cat_registrada_previsto,acidentes_tipicos_previsto,acidentes_trajeto_previsto,doenca_trabalho_previsto,sem_cat_registrada_previsto
8,910,2023,490,450,357,62,31,40,355,327,262,33,31,28
17,1921,2023,265,205,111,72,22,60,144,111,75,29,6,32
26,2513,2023,459,434,378,51,5,25,490,449,374,65,10,40
35,3321,2023,897,813,698,110,5,84,582,519,439,68,11,63
44,3520,2023,56,50,27,23,0,6,36,29,21,7,0,6


In [169]:
df_resultados_completos.shape

(10, 14)

In [170]:
df_resultados_completos = df_resultados_completos.iloc[:,[0,1,2,8,3,9,4,10,5,11,6,12,7,13]]
df_resultados_completos.head()

,cnae,ano,total_acidentes,total_acidentes_previsto,com_cat_registrada,com_cat_registrada_previsto,acidentes_tipicos,acidentes_tipicos_previsto,acidentes_trajeto,acidentes_trajeto_previsto,doenca_trabalho,doenca_trabalho_previsto,sem_cat_registrada,sem_cat_registrada_previsto
8,910,2023,490,355,450,327,357,262,62,33,31,31,40,28
17,1921,2023,265,144,205,111,111,75,72,29,22,6,60,32
26,2513,2023,459,490,434,449,378,374,51,65,5,10,25,40
35,3321,2023,897,582,813,519,698,439,110,68,5,11,84,63
44,3520,2023,56,36,50,29,27,21,23,7,0,0,6,6


## 4. Amazenamento no drive

In [171]:
# Define o caminho e o nome da pasta no Google Drive
caminho_drive = '/content/drive/MyDrive/dados/output/'

In [172]:
# Salva o Modelo Treinado
caminho_modelo = os.path.join(caminho_drive, 'modelo_random_forest_multi.joblib')
joblib.dump(modelo_multi, caminho_modelo)

print(f"Modelo salvo com sucesso em: {caminho_modelo}")

Modelo salvo com sucesso em: /content/drive/MyDrive/dados/output/modelo_random_forest_multi.joblib


In [173]:
# Salva o dicionário de métricas em JSON
caminho_metricas = os.path.join(caminho_drive, 'metricas_modelo.json')
with open(caminho_metricas, 'w') as f:
    json.dump(metricas, f, indent=4) # indent=4 para deixar o arquivo bem formatado

print(f"Métricas salvas com sucesso em: {caminho_metricas}")

Métricas salvas com sucesso em: /content/drive/MyDrive/dados/output/metricas_modelo.json


In [174]:
# Salva as previsões
file_path = f'{caminho_drive}results.csv'
df_resultados_completos.to_csv(file_path, index=False)

print(f"Arquivo foi salvo com sucesso em: {file_path}")

Arquivo foi salvo com sucesso em: /content/drive/MyDrive/dados/output/results.csv
